This notebook descrbies the PORT-EK pipeline analysis of k-mers genreated from the "deer" dataset from [reference to our paper].
To use it you will need the k-mer indices generated with PORTEKfind.py in the output/deer/15mer_indices directory.
To generate the indices as in the paper, you will need to run PORTEKfind.py on the appropriate GISAID data sets, previously downloaded in .fasta files, using the following commands from the main PORT-EK directory:
 - python PORTEKfind.py "input/deer/EPI_SET_240422va.fasta" "output/deer/15mer_indices/" --k 15 --group deer
 - python PORTEKfind.py "input/deer/EPI_SET_240422rw.fasta" "output/deer/15mer_indices/" --k 15 --group humearly  
 - python PORTEKfind.py "input/deer/EPI_SET_240422qc.fasta" "output/deer/15mer_indices/" --k 15 --group humlate  

For the selection of optimal k please see optimal_deer_k_selection.ipynb notebook. 
Note that GISAID web interface only allows downloading up to 10 000 sequences in one file, so when you download the 04.2021 and 11.2021 human viral sequences they will be in several files. You have to first combine them into a single .fasta file or run PORTEKfind separately on the partial files. 


1. Import necessary libraries and PORT-EK source code:

In [1]:
import sys
import json
import pathlib
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#adding portek source directory to sys path before importing
portek_path = "../portek"
sys.path.insert(0,portek_path)
import portek

pd.options.mode.copy_on_write = True

2. Declare data set specific definitions and functions:

In [2]:
#PORT-EK parameters
c = 0.01  # This is the conservation thershold used in k-mer rarity filter
m = 2  # This is the maximum number of mismatches allowed when re-examining rare k-mers
min_RMSE = 0.1  # This is the RMSE threshold used to select enriched k-mers
m_map = 2  # This is the maximum number of mismatches allowed when mapping k-mers to reference genome
l_map = 1000  # This the maximum allowed offset of mapping position from average position of k-mer in samples

#Relative path to k-mer indices
INPUT_PATH = "../output/deer/15mer_indices"

#Data set specific definitions of k-mer type, column names, reference gene and protein mapping, and colors for plots.

FREQ_COLS = ['deer_freq','humearly_freq','humlate_freq']
AVG_COLS = ['deer_avg','humearly_avg','humlate_avg']

VOLCANO_CMAP = {
    "not significant": ("#DDDDDD", 0.5),
    "deer enriched": ("#ffa401", 1),
    "human enriched": ("#005ff5", 1),
    "time-dependant": ("#b99e9e", 0.8),
}
GENE_ORDER_LIST = [
    "orf1ab",
    "S",
    "orf3a",
    "E",
    "M",
    "orf6",
    "orf7a",
    "orf7b",
    "orf8",
    "N",
    "orf10",
    "intergenic",
]
PROTEIN_ORDER_LIST = [
    "nsp1",
    "nsp2",
    "nsp3",
    "nsp4",
    "nsp5",
    "nsp6",
    "nsp7",
    "nsp8",
    "nsp9",
    "nsp10",
    "nsp12",
    "nsp13",
    "nsp14",
    "nsp15",
    "nsp16",
    "S",
    "orf3a",
    "E",
    "M",
    "orf6",
    "orf7a",
    "orf7b",
    "orf8",
    "N",
    "orf10",
    "non-coding",
]


def assign_kmer_type(row):
    if (
        row["deer_early_err"] > 0
        and row["deer_late_err"] > 0
        and row["p-value_late"] < 0.01
        and row["p-value_early"] < 0.01
    ):
        return "deer over-represented"
    elif (
        row["deer_early_err"] < 0
        and row["deer_late_err"] < 0
        and row["p-value_late"] < 0.01
        and row["p-value_early"] < 0.01
    ):
        return "human over-represented"
    elif row["p-value_early"] < 0.01 or row["p-value_late"] < 0.01:
        return "time-dependant"
    else:
        return "not significant"


def assign_gene(pos):
    if pos in range(266, 21555 + 1):
        gene = "orf1ab"
    elif pos in range(21563, 25384 + 1):
        gene = "S"
    elif pos in range(25393, 26220 + 1):
        gene = "orf3a"
    elif pos in range(26245, 26472 + 1):
        gene = "E"
    elif pos in range(26523, 27191 + 1):
        gene = "M"
    elif pos in range(27202, 27387 + 1):
        gene = "orf6"
    elif pos in range(27394, 27759 + 1):
        gene = "orf7a"
    elif pos in range(27756, 27887 + 1):
        gene = "orf7b"
    elif pos in range(27894, 28259 + 1):
        gene = "orf8"
    elif pos in range(28274, 29533 + 1):
        gene = "N"
    elif pos in range(29558, 29674 + 1):
        gene = "orf10"
    else:
        gene = "intergenic"
    return gene


def assign_protein(pos):
    if pos in range(266, 805 + 1):
        protein = "nsp1"
    elif pos in range(806, 2719 + 1):
        protein = "nsp2"
    elif pos in range(2720, 8554 + 1):
        protein = "nsp3"
    elif pos in range(8555, 10054 + 1):
        protein = "nsp4"
    elif pos in range(10055, 10972 + 1):
        protein = "nsp5"
    elif pos in range(10973, 11842 + 1):
        protein = "nsp6"
    elif pos in range(11843, 12091 + 1):
        protein = "nsp7"
    elif pos in range(12092, 12685 + 1):
        protein = "nsp8"
    elif pos in range(12686, 13024 + 1):
        protein = "nsp9"
    elif pos in range(13025, 13441 + 1):
        protein = "nsp10"
    elif pos in range(13442, 16236 + 1):
        protein = "nsp12"
    elif pos in range(16237, 18039 + 1):
        protein = "nsp13"
    elif pos in range(18040, 19620 + 1):
        protein = "nsp14"
    elif pos in range(19621, 20658 + 1):
        protein = "nsp15"
    elif pos in range(20659, 21555 + 1):
        protein = "nsp16"
    elif pos in range(21563, 25384 + 1):
        protein = "S"
    elif pos in range(25393, 26220 + 1):
        protein = "orf3a"
    elif pos in range(26245, 26472 + 1):
        protein = "E"
    elif pos in range(26523, 27191 + 1):
        protein = "M"
    elif pos in range(27202, 27387 + 1):
        protein = "orf6"
    elif pos in range(27394, 27759 + 1):
        protein = "orf7a"
    elif pos in range(27756, 27887 + 1):
        protein = "orf7b"
    elif pos in range(27894, 28259 + 1):
        protein = "orf8"
    elif pos in range(28274, 29533 + 1):
        protein = "N"
    elif pos in range(29558, 29674 + 1):
        protein = "orf10"
    else:
        protein = "non-coding"
    return protein

3. Construct k-mer count matrix and apply rarity filter

In [3]:
kmer_set = set()
sample_list = []
in_path = pathlib.Path(INPUT_PATH).glob('**/*')

for filename in in_path:
    sample_list.append(filename.stem)
    with open(filename, mode="r") as in_file:
        temp_dict = json.load(in_file)
    kmer_set.update(temp_dict.keys())

all_kmer_matrix = pd.DataFrame(0, index=list(kmer_set), columns=sample_list, dtype="uint8")
deer_sample_idx = [sample for sample in all_kmer_matrix.columns if "deer" in sample]
humearly_sample_idx = [sample for sample in all_kmer_matrix.columns if "humearly" in sample]
humlate_sample_idx = [sample for sample in all_kmer_matrix.columns if "humlate" in sample]

print(f"\nImported {len(kmer_set)} kmers and {len(sample_list)} samples.")

counter = 1
tot_files = len(sample_list)
in_path = pathlib.Path(INPUT_PATH).glob('**/*')

for filename in in_path:
    with open(filename, mode="r") as in_file:
        temp_dict = json.load(in_file)
    count_dict = {f"{filename.stem}":[len(pos) for pos in temp_dict.values()]}
    temp_df = pd.DataFrame(count_dict, index=temp_dict.keys(), dtype="uint8")
    all_kmer_matrix.update(temp_df)
    print(f"Completed {filename.stem}. {counter} of {tot_files} indices done.", end="\r", flush=True)
    counter += 1

bin_kmer_matrix = all_kmer_matrix > 0
all_kmer_matrix['deer_freq'] = bin_kmer_matrix.loc[:,deer_sample_idx].mean(axis=1)
all_kmer_matrix['humearly_freq'] = bin_kmer_matrix.loc[:,humearly_sample_idx].mean(axis=1)
all_kmer_matrix['humlate_freq'] = bin_kmer_matrix.loc[:,humlate_sample_idx].mean(axis=1)
all_kmer_matrix['deer_avg'] = all_kmer_matrix.loc[:,deer_sample_idx].mean(axis=1)
all_kmer_matrix['humearly_avg'] = all_kmer_matrix.loc[:,humearly_sample_idx].mean(axis=1)
all_kmer_matrix['humlate_avg'] = all_kmer_matrix.loc[:,humlate_sample_idx].mean(axis=1)
del bin_kmer_matrix

if 'AAAAAAAAAAAAAAA' in all_kmer_matrix.index:
    all_kmer_matrix = all_kmer_matrix.drop('AAAAAAAAAAAAAAA')

common_kmer_matrix = portek.filter_kmers(all_kmer_matrix, freq_cols=FREQ_COLS, cons_thr=c)

print(f"\n{len(common_kmer_matrix)} common k-mers remaining after filtering at a threshold of {c}.")


Imported 371342 kmers and 33767 samples.
Completed humearly_EPI_ISL_1636116. 33767 of 33767 indices done..
41946 kmers remaining after filtering at a threshold of 0.01.
Finished importing count matrices and calculating frequencies.
Total matrix size is 371342 kmers by 33767 samples.


4. Calculate common k-mer statistics and identify host over-represented k-mers.

In [ ]:
common_kmer_matrix['deer_early_err'] = common_kmer_matrix['deer_avg']-common_kmer_matrix['humearly_avg']
common_kmer_matrix['deer_late_err'] = common_kmer_matrix['deer_avg']-common_kmer_matrix['humlate_avg']
common_kmer_matrix['deer_RMSE'] =  np.sqrt(((common_kmer_matrix['deer_early_err'])**2+(common_kmer_matrix['deer_late_err'])**2)/2)
common_kmer_matrix['seq'] = common_kmer_matrix.index
common_kmer_matrix['p-value_early'] = common_kmer_matrix['seq'].apply(portek.calc_kmer_pvalue,args=(deer_sample_idx, humearly_sample_idx, common_kmer_matrix))
common_kmer_matrix['p-value_late'] = common_kmer_matrix['seq'].apply(portek.calc_kmer_pvalue,args=(deer_sample_idx, humlate_sample_idx, common_kmer_matrix))
common_kmer_matrix = common_kmer_matrix.sort_values('deer_RMSE',ascending=False)
common_kmer_matrix = common_kmer_matrix.drop('seq', axis=1)
common_kmer_matrix['-log10_p-value_early'] = -np.log10(common_kmer_matrix['p-value_early'])
common_kmer_matrix['-log10_p-value_late'] = -np.log10(common_kmer_matrix['p-value_late'])
common_kmer_matrix['group'] = common_kmer_matrix.apply(assign_kmer_type, axis=1)
common_kmer_stat_matrix = common_kmer_matrix.drop(sample_list, axis=1)
deer_overrep = common_kmer_matrix[common_kmer_matrix['group'] == 'deer over-represented']
human_overrep = common_kmer_matrix[common_kmer_matrix['group'] == 'human over-represented']

Optional: save/load k-mer count and statistics matrices. Uncomment the relevant lines to save/load particular files.

In [ ]:
# common_kmer_matrix.to_csv("output/deer/common_15mer_count_matrix.csv")
# common_kmer_stat_matrix.to_csv("output/deer/common_15mer_stat_matrix.csv")
# all_kmer_matrix.to_csv("output/deer/all_15mer_count_matrix.csv")

# common_kmer_matrix = pd.read_csv('output/deer/common_15mer_count_matrix.csv', index_col=0)
# all_kmer_matrix = pd.read_csv('output/deer/all_15mer_count_matrix.csv', index_col=0)
# common_kmer_stat_matrix = pd.read_csv("output/deer/common_15mer_stat_matrix.csv", index_col=0)

5. Visualize common k-mers using volcano plots. Uncomment the last line in each cell to save as svg.

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data=common_kmer_matrix, x='deer_early_err', y = '-log10_p-value_early', s=10, linewidth = 0, hue='group', palette=VOLCANO_CMAP)
plt.axhline(y=-np.log10(0.01), color = 'black')
ax.set_xlim(-1,1)
ax.set_title('Average kmer count change vs early human samples')
ax.set_xlabel('Average kmer count change')
ax.set_ylabel('-log10 of p-value')
handles, labels = plt.gca().get_legend_handles_labels()
order = [1,0,2,3]
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order])
# plt.savefig("output/deer/15mers_change_vs_p_early.svg", dpi = 600, format = "svg")

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data=common_kmer_matrix, x='deer_late_err', y = '-log10_p-value_late',s=10, linewidth = 0, hue='group', palette=VOLCANO_CMAP)
plt.axhline(y=-np.log10(0.01), color = 'black')
ax.set_xlim(-1,1)
ax.set_title('Average kmer count change vs late human samples')
ax.set_xlabel('Average kmer count change')
ax.set_ylabel('-log10 of p-value')
handles, labels = plt.gca().get_legend_handles_labels()
order = [1,0,2,3]
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order])
# plt.savefig("output/deer/change_vs_p_late.svg", dpi = 600, format = "svg")

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data = common_kmer_matrix, x='deer_early_err', y='deer_late_err',s=10, linewidth = 0, hue='group', palette = VOLCANO_CMAP)
plt.axline((0, 0), slope=1, color=("black",0.5), linestyle="dashed")
ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
ax.set_title('Average kmer count change vs early and late human samples')
ax.set_xlabel('Average kmer count change vs early human samples')
ax.set_ylabel('Average kmer count change vs late human samples')
handles, labels = plt.gca().get_legend_handles_labels()
order = [1,0,2,3]
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order])
# plt.savefig("output/deer/change_corr.svg", dpi = 600, format = "svg")

6. Optionally re-examin rare k-mers similar to over-represented k-mers. Run the two cells below only if you want to include potential rare, but still enriched k-mers in the analysis.

In [ ]:
deer_overrep_rare = all_kmer_matrix.loc[(all_kmer_matrix['deer_avg'] == all_kmer_matrix[AVG_COLS].max(axis=1))]
deer_overrep_rare = deer_overrep_rare[~deer_overrep_rare.index.isin(deer_overrep.index)]

#Comment the following line to avoid lengthy k-mer similarity graph calculation
deer_overrep_similarity_graph = portek.build_similarity_graph_two_list(deer_overrep.index.tolist(), deer_overrep_rare.index.tolist(),m)

#Uncomment the first of the following blocks to save the graph as pickle file. Uncomment the second to load a previously calculated graph.

# with open(f"output/deer/dor_15mer_{m}m.pickle", mode = "wb") as out_file:
#     pickle.dump(deer_overrep_similarity_graph, out_file)

# with open(f"output/deer/dor_15mer_{m}m.pickle", mode = "rb") as in_file:
#     deer_overrep_similarity_graph = pickle.load(in_file)

deer_overrep_rare = deer_overrep_rare[deer_overrep_rare.index.isin(deer_overrep_similarity_graph.nodes)]

deer_overrep_rare['deer_early_err'] = deer_overrep_rare['deer_avg']-deer_overrep_rare['humearly_avg']
deer_overrep_rare['deer_late_err'] = deer_overrep_rare['deer_avg']-deer_overrep_rare['humlate_avg']
deer_overrep_rare['deer_RMSE'] =  np.sqrt(((deer_overrep_rare['deer_early_err'])**2+(deer_overrep_rare['deer_late_err'])**2)/2)
deer_overrep_rare['seq'] = deer_overrep_rare.index
deer_overrep_rare['p-value_early'] = deer_overrep_rare['seq'].apply(portek.calc_kmer_pvalue,args=(deer_sample_idx, humearly_sample_idx, deer_overrep_rare))
deer_overrep_rare['p-value_late'] = deer_overrep_rare['seq'].apply(portek.calc_kmer_pvalue,args=(deer_sample_idx, humlate_sample_idx, deer_overrep_rare))
deer_overrep_rare = deer_overrep_rare.sort_values('deer_RMSE',ascending=False)
deer_overrep_rare = deer_overrep_rare.drop('seq', axis=1)
deer_overrep_rare['-log10_p-value_early'] = -np.log10(deer_overrep_rare['p-value_early'])
deer_overrep_rare['-log10_p-value_late'] = -np.log10(deer_overrep_rare['p-value_late'])
deer_overrep_rare['group'] = deer_overrep_rare.apply(assign_kmer_type, axis=1)
deer_overrep_rare  = deer_overrep_rare[deer_overrep_rare['group'] == 'deer over-represented']
deer_overrep = pd.concat([deer_overrep_rare, deer_overrep])

In [ ]:
human_overrep_rare = all_kmer_matrix.loc[(all_kmer_matrix['deer_avg'] == all_kmer_matrix[AVG_COLS].min(axis=1))]
human_overrep_rare = human_overrep_rare[~human_overrep_rare.index.isin(human_overrep.index)]

#Comment the following line to avoid lengthy k-mer similarity graph calculation
human_overrep_similarity_graph = portek.build_similarity_graph_two_list(human_overrep.index.tolist(), human_overrep_rare.index.tolist(),m)

#Uncomment the first of the following blocks to save the graph as pickle file. Uncomment the second to load a previously calculated graph.

# with open(f"output/deer/hor_15mer_{m}m.pickle", mode = "wb") as out_file:
#     pickle.dump(human_overrep_similarity_graph, out_file)

# with open(f"output/deer/hor_15mer_{m}m.pickle", mode = "rb") as in_file:
#     human_overrep_similarity_graph = pickle.load(in_file)

human_overrep_rare = human_overrep_rare[deer_overrep_rare.index.isin(deer_overrep_similarity_graph.nodes)]

human_overrep_rare['deer_early_err'] = human_overrep_rare['deer_avg']-human_overrep_rare['humearly_avg']
human_overrep_rare['deer_late_err'] = human_overrep_rare['deer_avg']-human_overrep_rare['humlate_avg']
human_overrep_rare['deer_RMSE'] =  np.sqrt(((human_overrep_rare['deer_early_err'])**2+(human_overrep_rare['deer_late_err'])**2)/2)
human_overrep_rare['seq'] = human_overrep_rare.index
human_overrep_rare['p-value_early'] = human_overrep_rare['seq'].apply(portek.calc_kmer_pvalue,args=(deer_sample_idx, humearly_sample_idx, human_overrep_rare))
human_overrep_rare['p-value_late'] = human_overrep_rare['seq'].apply(portek.calc_kmer_pvalue,args=(deer_sample_idx, humlate_sample_idx, human_overrep_rare))
human_overrep_rare = human_overrep_rare.sort_values('deer_RMSE',ascending=False)
human_overrep_rare = human_overrep_rare.drop('seq', axis=1)
human_overrep_rare['-log10_p-value_early'] = -np.log10(human_overrep_rare['p-value_early'])
human_overrep_rare['-log10_p-value_late'] = -np.log10(human_overrep_rare['p-value_late'])
human_overrep_rare['group'] = human_overrep_rare.apply(assign_kmer_type, axis=1)
human_overrep_rare  = human_overrep_rare[human_overrep_rare['group'] == 'human over-represented']
human_overrep = pd.concat([human_overrep_rare, deer_overrep])

7. Find enriched k-mers by applying the RMSE filter. Export enriched k-mers for use in host classification.

In [ ]:
deer_enriched = deer_overrep[deer_overrep['deer_RMSE']>min_RMSE]
human_enriched = human_overrep[human_overrep['deer_RMSE']>min_RMSE]


def assign_host_numerical(sample_id):
    if sample_id in deer_sample_idx:
        host = 1
    else:
        host = 0
    return host


counts_for_classifier = pd.concat([deer_enriched, human_enriched]).T
counts_for_classifier.drop(common_kmer_stat_matrix.columns, axis=0, inplace=True)
counts_for_classifier['host'] = counts_for_classifier.index.map(assign_host_numerical)

counts_for_classifier.to_csv("output/deer/15mer_counts_for_classifier.csv")

8. Map enriched k-mers to the reference genome and identify host-enriched mutations.